In [5]:
%run tree.py
%run meanvar_tree_utilities.py

In [6]:
N_list = [100, 200, 400]; p = 10
R = 0.1; alpha = 0.1; obj_coef = 0
lb = 0; ub = 1;  sum_bound = 1; if_stoch_constr = False
runs = 10

generate_Y = generate_Y_lognormal
cond_mean = [lambda x: np.exp(x[:, 0])/5, lambda x: x[:, 0]/5, lambda x: np.abs(x[:, 0])/5]
cond_std = [lambda x: 1 - 0.5*((-3<=x[:, 1]) & (x[:, 1]<=-1)), lambda x: 1 - 0.5*((-1<=x[:, 1])&(x[:, 1]<=1)), lambda x: 1 - 0.5*((1<=x[:, 1])&(x[:, 1]<=3))]

opt_solver = partial(solve_mean_variance, alpha = alpha, R = R, obj_coef = obj_coef, lb = lb, ub = ub, sum_bound = sum_bound, if_stoch_constr = if_stoch_constr)
hessian_computer = partial(compute_hessian, alpha = alpha)
active_constraint = partial(search_active_constraint,  R = R, lb = lb, ub = ub, sum_bound = sum_bound, if_stoch_constr = if_stoch_constr)
gradient_computer = partial(compute_gradient,  alpha = alpha, R = R, obj_coef = obj_coef)
update_step = partial(compute_update_step, R = R)

time_list = {str(N): {key: np.zeros(runs) for key in ["rf_approx_risk", "rf_approx_sol", "rf_rf", "rf_oracle"]} for N in N_list}

In [9]:
for N in N_list:
    print("N:", N)
    n_proposals = N; 
    mtry = p;
    subsample_ratio = 1;
    max_depth=100; 
    min_leaf_size=10; 
    balancedness_tol = 0.2; 
    honesty = False;
    verbose = False; oracle = True;
    bootstrap = True; 
    
    X_list = [np.random.normal(size = (N, p)) for run in range(runs)]
    Y_list = [generate_Y(X_list[run], cond_mean, cond_std) for run in range(runs)]

    for run in range(runs):
        print("run:", run)
        Y = Y_list[run]; Y_est = Y_list[run]
        X = X_list[run]; X_est = X_list[run]; 
        
        time1 = time.time()
        rf_approx_risk = build_tree(Y, X, Y_est, X_est, 
                             opt_solver = opt_solver, hessian_computer = hessian_computer,
                             gradient_computer = gradient_computer, 
                             search_active_constraint = active_constraint,
                             compute_update_step = update_step,
                             crit_computer = compute_crit_approx_risk, 
                             honesty = honesty, mtry = mtry,
                             min_leaf_size = min_leaf_size, max_depth = max_depth, 
                             n_proposals = n_proposals, balancedness_tol = balancedness_tol,
                             verbose = verbose)
        time2 = time.time()
        time_list[str(N)]["rf_approx_risk"][run] = time2 - time1

        time1 = time.time()
        rf_approx_sol = build_tree(Y, X, Y_est, X_est, 
                             opt_solver = opt_solver, hessian_computer = hessian_computer,
                             gradient_computer = gradient_computer, 
                             search_active_constraint = active_constraint,
                             compute_update_step = update_step,
                             crit_computer = partial(compute_crit_approx_sol, obj_coef = obj_coef, alpha = alpha), 
                             honesty = honesty, mtry = mtry,
                             min_leaf_size = min_leaf_size, max_depth = max_depth, 
                             n_proposals = n_proposals, balancedness_tol = balancedness_tol,
                             verbose = verbose)
        time2 = time.time()
        time_list[str(N)]["rf_approx_sol"][run] = time2 - time1

        time1 = time.time()
        rf_rf = build_tree(Y, X, Y_est, X_est, 
                             opt_solver = opt_solver, hessian_computer = hessian_computer,
                             gradient_computer = gradient_computer, 
                             search_active_constraint = active_constraint,
                             compute_update_step = update_step,
                             crit_computer = compute_crit_rf, 
                             honesty = honesty, mtry = mtry,
                             min_leaf_size = min_leaf_size, max_depth = max_depth, 
                             n_proposals = n_proposals, balancedness_tol = balancedness_tol,
                             verbose = verbose)
        time2 = time.time()
        time_list[str(N)]["rf_rf"][run] = time2 - time1
    
        time1 = time.time()
        rf_oracle = build_tree(Y, X, Y_est, X_est, 
                             opt_solver = opt_solver, hessian_computer = hessian_computer,
                             gradient_computer = gradient_computer, 
                             search_active_constraint = active_constraint,
                             compute_update_step = update_step,
                             crit_computer = partial(compute_crit_oracle, solver = opt_solver), 
                             honesty = honesty, mtry = mtry,
                             min_leaf_size = min_leaf_size, max_depth = max_depth, 
                             n_proposals = n_proposals, balancedness_tol = balancedness_tol,
                             verbose = verbose)
        time2 = time.time()
        time_list[str(N)]["rf_oracle"][run] = time2 - time1
    

N: 100
run: 0
run: 1
run: 2
run: 3
run: 4
run: 5
run: 6
run: 7
run: 8
run: 9
N: 200
run: 0
run: 1
run: 2
run: 3
run: 4
run: 5
run: 6
run: 7
run: 8
run: 9
N: 400
run: 0
run: 1
run: 2
run: 3
run: 4
run: 5
run: 6
run: 7
run: 8
run: 9


In [13]:
pickle.dump(time_list, open("time_meanvar.pkl", "wb"))

In [11]:
{N: {m: time_list[N][m].mean() for m in time_list[N].keys()} for N in time_list.keys()}

{'100': {'rf_approx_risk': 0.09544167518615723,
  'rf_approx_sol': 0.08060789108276367,
  'rf_oracle': 10.244396829605103,
  'rf_rf': 0.07557811737060546},
 '200': {'rf_approx_risk': 0.22738821506500245,
  'rf_approx_sol': 0.28768579959869384,
  'rf_oracle': 32.584637665748595,
  'rf_rf': 0.19053428173065184},
 '400': {'rf_approx_risk': 0.7578061580657959,
  'rf_approx_sol': 1.4811426162719727,
  'rf_oracle': 90.39753568172455,
  'rf_rf': 0.6741846323013305}}

In [12]:
{N: {m: time_list[N][m].std() for m in time_list[N].keys()} for N in time_list.keys()}

{'100': {'rf_approx_risk': 0.040564479592028024,
  'rf_approx_sol': 0.014451931200771257,
  'rf_oracle': 1.5058775012916399,
  'rf_rf': 0.02536320911336496},
 '200': {'rf_approx_risk': 0.04051650062988821,
  'rf_approx_sol': 0.05279786884238977,
  'rf_oracle': 7.631760692418332,
  'rf_rf': 0.0367482486934303},
 '400': {'rf_approx_risk': 0.08954796084947915,
  'rf_approx_sol': 0.5078014482022202,
  'rf_oracle': 13.674538794130605,
  'rf_rf': 0.20004046622068203}}